In [22]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from itertools import product

In [19]:
def match_images(image1, image2, show_matches=False, nfeatures=1000,scaleFactor = 1.2, nlevels=8):
    # Load color images
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    
    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=nfeatures,scaleFactor=scaleFactor,nlevels=nlevels )

    # Find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    # print(len(des1))
    
    
    if des2 is None or des1 is None:
        # print(type(des2))
        return 0    

    # Use the BFMatcher to find the best matches
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # print(f"lem{len(matches)}")
    # print(matches)
    # Apply ratio test
    good_matches = []

    for match in matches:
        # Check if there are enough values to unpack
        if len(match) == 2:
            m, n = match
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

    # Display the matches
    if show_matches:
        # Draw matches on the images
        img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        cv2.imshow("Matches", img_matches)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return len(good_matches)

def find_total_matches(folder_path, timestamps, show_matches=False, nfeatures=1000,scaleFactor = 1.2, nlevels=8):
    
    total_matches = 0
    num_of_matches_list = []
    for i in tqdm(range(len(timestamps) - 1)):
        current_image = os.path.join(folder_path, timestamps[i])+'.png'
        next_image = os.path.join(folder_path, timestamps[i + 1])+'.png'
        
        # Match features and accumulate the total matches
        matches = match_images(current_image, next_image,show_matches,nfeatures=1000,scaleFactor = 1.2, nlevels=8)
        total_matches += matches
        num_of_matches_list.append(matches)

    
    print(f"Total matches: {total_matches}")

    return num_of_matches_list, total_matches,




In [20]:
# nfeatures=3000
# scaleFactor = 1.05
# nlevels=20
# Provide the path to the folder containing images
def evaluate_orb_config(nfeatures=3000, scaleFactor = 1.05, nlevels=20,show_plot=False):
    time_stamps = "/home/ayon/git/orbslam3_docker/Datasets/EuRoC/time_stamp.txt"

    df = pd.read_csv(time_stamps, header=None)
    time_stamps = df[0].to_numpy().astype('str')
    match_list_df = pd.DataFrame()


    if show_plot:
        plt.figure(figsize=(16,12))
    total_time_list = []
    for i in range(1,7):
        print(f"matching for MH0{i}")
        image_folder = f"/home/ayon/git/orbslam3_docker/Datasets/EuRoC/MH0{i}/mav0/cam0/data"
        assert len(os.listdir(image_folder))==len(time_stamps)
        start_time = time.time()
        num_of_matches_list, total_matches = find_total_matches(image_folder, time_stamps,nfeatures=nfeatures,scaleFactor = scaleFactor, nlevels=nlevels)
        end_time = time.time()
        total_time_list.append(end_time-start_time)
        
        match_list_df[f'MH0{i}'] = num_of_matches_list
        if show_plot:
            plt.plot(num_of_matches_list, label=f'MH0{i}')

    match_list_df.to_csv(f'orb_matching_results/{nfeatures}_{scaleFactor}_{nlevels}.csv')
    np.savetxt(f'orb_matching_results/times_{nfeatures}_{scaleFactor}_{nlevels}.txt', my_list, fmt='%d', delimiter=', ')
    
    if show_plot:
        plt.legend()
        plt.show()

In [24]:
nfeatures_list = [1000, 2000, 3000, 4000]
scaleFactor_list = [1.2, 1.15, 1.10, 1.05]
nlevels_list = [8,12,16,20]

# Get all possible combinations
combinations = list(product(nfeatures_list, scaleFactor_list, nlevels_list))

# Print the result
for nfeatures, scaleFactor, nlevels in tqdm(combinations):
    evaluate_orb_config(nfeatures=nfeatures, scaleFactor = scaleFactor, nlevels=nlevels, show_plot=False)
    

SyntaxError: invalid syntax (<ipython-input-24-b5c4e4bdf617>, line 10)

-2.384185791015625e-07